In [ ]:
import cv2
from tracker import *

#Create Tracker Object 
tracker = EuclideanDistTracker()


cap = cv2.VideoCapture("highway.mp4")

#Object detection from the stable camera
object_detector = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=40)

while True:
    
    ret, frame = cap.read()
    
    height, width, _ = frame.shape
    # print(height, width)
    
    #Extract Region of interest
    roi = frame[340:720,500:800]
    
    
    # 1 - Object Detection
    
    mask = object_detector.apply(roi)
    _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY) # TO reove the gray area from the mask
    (_,contours,_) = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    detections = []
    for cnt in contours:
        
        #calculate area and remove small elements
        area = cv2.contourArea(cnt)
        
        if area > 100 :
            #cv2.drawContours(roi, [cnt], -1, (150,255,0), 2)
            x,y,w,h = cv2.boundingRect(cnt)
            cv2.rectangle(roi, (x,y), (x+w, y+h), (255,0,255), 2)
            #print(x,y,w,h)
            detections.append([x,y,w,h])
    
    
    # 2 - Object Tracking
    
    boxes_ids = tracker.update(detections)
    for box_id in boxes_ids:
        x,y,w,h,id = box_id
        cv2.putText(roi, str(id), (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,200), 3)
        cv2.rectangle(roi, (x,y), (x+w, y+h), (255,0,255), 2)
        
    
    
    # print(detections)
    print(boxes_ids)
    cv2.imshow("Roi", roi)
    cv2.imshow("Mask", mask)
    cv2.imshow("Frame", frame)
    
    
    key = cv2.waitKey(30)
    if key == 27:
        break #esc key to quit
        
    
    
        
cap.release()
cv2.destroyAllWindows()

In [ ]:
cv2.putText??